In [5]:
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from langchain.tools.retriever import create_retriever_tool
from langchain.agents import create_openai_tools_agent


apiwrap_arxiv=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=500)
arxi=ArxivQueryRun(api_wrapper=apiwrap_arxiv)
arxi.name

'arxiv'

In [6]:
apiwrap_wiki=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=500)
wiki=WikipediaQueryRun(api_wrapper=apiwrap_wiki)
wiki.name

'wikipedia'

In [7]:
import os
from dotenv import load_dotenv
load_dotenv()
groq_api_key=os.getenv("GROQ_API_KEY")
hf_key=os.getenv("HF_KEY")
llm=ChatGroq(api_key=groq_api_key,model="llama-3.3-70b-versatile")

In [8]:
loader=WebBaseLoader("https://en.wikipedia.org/wiki/Superman_(2025_film)")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=documents.split_documents(docs)
vectorstoredb=FAISS.from_documents(documents=documents,embedding=HuggingFaceEmbeddings())
retriever=vectorstoredb.as_retriever()
retriever_tool=create_retriever_tool(retriever,"About superman","Search anything about superman")

In [9]:
tools=[wiki,arxi,retriever_tool]

In [10]:
prompt=ChatPromptTemplate(
    [
        ("system","You are a useful assistant. Be mindful of the user"),
        ("user","{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad")
        
    ]
)
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a useful assistant. Be mindful of the user'), additional_kwargs={}),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [11]:
agent=create_openai_tools_agent(llm=llm,tools=tools,prompt=prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'agent_scratchpad': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='

In [12]:
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'agent_scratchpad': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typin

In [14]:
agent_executor.invoke({"input":"Summarise the plot of super man "})




> Entering new AgentExecutor chain...

Invoking: `About superman` with `{'query': 'superman plot summary'}`


Superman agrees to be interviewed by Lois Lane, his girlfriend and reporter colleague at the Daily Planet, leading to an argument when Lois presses him on the legal and ethical implications of his actions in Boravia. Luthor unleashes a kaiju on Metropolis as a distraction while he infiltrates the Fortress with Ultraman and the Engineer. They overpower the robots, capture Krypto, and discover the Kryptonian message; the Engineer restores the damaged portion. Superman rescues civilians while fighting the kaiju with the "Justice Gang", a team composed of heroes Green Lantern, Mister Terrific, and Hawkgirl. Despite Superman wanting to take the creature alive, Mister Terrific kills it. Luthor broadcasts the Kryptonian message and Superman is shocked to hear the second half, in which his parents urge him to conquer Earth and take many wives to restore the Kryptonian race.

Lois and

KeyboardInterrupt: 